# 使用 OpenAI 接口完成主题建模和情感分析任务

In [ ]:
import os
import pandas as pd
from openai import OpenAI
from rich import print as pp
from tqdm import tqdm
from pandarallel import pandarallel

# 1. 准备环境

## 1.1. 初始化 OpenAI 客户端

In [2]:
# 尝试阅读环境变量
try:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    if OPENAI_API_KEY is None:
        raise KeyError
    pp("[green]环境变量 OPENAI_API_KEY 已成功读取。")
except KeyError:
    OPENAI_API_KEY = None
    pp("[red]未找到环境变量 OPENAI_API_KEY。请确保已正确设置。")

client = OpenAI(api_key=OPENAI_API_KEY)  # 替换为你的 API 密钥

环境变量 OPENAI_API_KEY 已成功读取。

In [3]:
def topic_modeling(text: str) -> str:
    """
    根据给定文本使用 GPT-4 模型进行主题建模
    
    参数:
        text (str): 需要进行主题建模的文本
    
    返回:
        str: GPT-4 模型返回的主题列表，多个主题用逗号分隔
    """
    # 构建主题建模提示
    system_prompt = "你是一个专业的主题建模助手，能够准确识别文本中包含的主要主题。请分析文本并从['政治','娱乐']两个类别中返回最相关的一个主题。如果无法确定，请给出最合理的猜测。请只返回结果。"
    user_prompt = f"请对下面的文本进行主题建模：\n\n\"{text}\"\n\n主题:"
    
    # 调用 GPT-4 模型
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 提取并返回结果
    return response.choices[0].message.content.strip()


# 3. 大规模处理

## 3.1. 结合 `tqdm` 显示进度条

In [ ]:
# 假定我们有共为 3 个的一组文本
texts = [
    "中国国防部召开记者会",
    "霉霉在 TikTok 开设账号,吸引年轻人",
    "总统发表重要讲话，讨论经济政策"]

# 对每个文本进行主题建模,但我们现在使用循环来处理,并展示处理进度
for text in tqdm(texts):
# ----------++++--------
# ----------在这里-------
    topic = topic_modeling(text) # 调用主题建模函数
    pp(f"[blue]文本:[/blue] {text}\n[green]主题:[/green] {topic}") # 输出结果

  0%|          | 0/3 [00:00<?, ?it/s]

文本: 中国国防部召开记者会
主题: 政治

 33%|███▎      | 1/3 [00:01<00:02,  1.04s/it]

文本: 霉霉在 TikTok 开设账号,吸引年轻人
主题: 娱乐

 67%|██████▋   | 2/3 [00:01<00:00,  1.22it/s]

文本: 总统发表重要讲话，讨论经济政策
主题: 政治

100%|██████████| 3/3 [00:02<00:00,  1.28it/s]


## 3.2. 结合 `pandarallel` 并行处理

In [5]:
# 初始化 pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=3)

# 将文本列表转换为 DataFrame 并并行处理
df = pd.DataFrame({
    "text": texts
})

# 应用并行主题建模
df["topic"] = df["text"].parallel_apply(topic_modeling)
# -----------------------++++++++++++++----------------
# ---------------------------在这里---------------------

# 显示结果
df

INFO: Pandarallel will run on 3 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


,text,topic
0,中国国防部召开记者会,政治
1,"霉霉在 TikTok 开设账号,吸引年轻人",娱乐
2,总统发表重要讲话，讨论经济政策,政治
